In [ ]:
! pip install pandas numpy matplotlib xlrd openpyxl
! pip install scikit-learn seaborn yellowbrick statsmodels prophet
! pip install ipywidgets --upgrade
! pip install jupyter --upgrade

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet

In [ ]:
var = pd.read_csv('../variable_descriptions.csv')
var

In [84]:
data = pd.read_csv("../data9724.csv.gz", compression="gzip")

In [87]:
data.drop('ds', axis=1, inplace=True)

In [ ]:
data['DATATIME']= pd.to_datetime(data['ANO'].astype(str) + data['DIAJ'].astype(str),format='%Y%j')
data.set_index("DATATIME",inplace=True)
data

In [ ]:
data.index.name = 'ds'
data

In [ ]:
for i in ['TAR','UR','CHUVA']:
    if i == 'CHUVA':
        df_c = data.drop(data[data['CHUVA']<0].index)
        aux = df_c[df_c['CHUVA']>0][i]
        print(aux.head())
    else:
        # Calcula Q1, Q3 e IQR
        Q1 = data[i].quantile(0.25)
        Q3 = data[i].quantile(0.75)
        IQR = Q3 - Q1

        # Define os limites
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Filtra os valores dentro dos limites
        aux = data[(data[i] >= lower_bound) & (data[i] <= upper_bound)][i]
    
    aux = aux.reset_index()
    df = aux.resample('D', on='ds').mean().reset_index()
    df_test = aux.resample('YE', on='ds').mean().reset_index()
    df.columns = ['ds','y']
    # Criar modelo Prophet com sazonalidades diárias e anuais
    model = Prophet(interval_width=0.95)  
    model.add_seasonality(name='daily', period=1, fourier_order=10)
    # model.add_seasonality(name='weekend_effect', period=7, fourier_order=5)
    model.add_seasonality(name='yearly', period=365.25, fourier_order=20)

    # Treinar o modelo
    model.fit(df)
    # Fazer previsões para todo o período histórico
    future = model.make_future_dataframe(periods=0, freq='D')  # Apenas prever dados passados
    forecast = model.predict(future)
    # Adicionar previsões ao DataFrame original
    df['yhat'] = forecast['yhat']
    df['residual'] = df['y'] - df['yhat']
    # Definir limites para eventos extremos
    limite_superior = df['residual'].mean() + 2 * df['residual'].std()
    limite_inferior = df['residual'].mean() - 2 * df['residual'].std()
    # Filtrar eventos extremos
    eventos_extremos = df[(df['residual'] > limite_superior) | (df['residual'] < limite_inferior)]
    # Criar coluna com ano dos eventos extremos
    eventos_extremos['year'] = eventos_extremos['ds'].dt.year
    # Contar eventos extremos por ano
    frequencia_eventos = eventos_extremos.groupby('year').size().reset_index(name='num_eventos')
    frequencia_eventos['DATATIME'] = pd.to_datetime(frequencia_eventos['year'],format='%Y')
    # SALVAR tabela com a frequência anual de eventos extremos
    frequencia_eventos.to_csv("../data9724"+f"_{i}"+".csv.gz", compression="gzip")
    
    # Preparar os dados para o Prophet
    df_pro = frequencia_eventos.reset_index().rename(columns={'DATATIME': 'ds', 'num_eventos': 'y'})
    # Inicializar o modelo
    model = Prophet()
    # Ajustar o modelo
    model.fit(df_pro)
    # Criar um DataFrame para datas futuras
    future = model.make_future_dataframe(periods=10, freq='Y')
    # Fazer previsões
    forecast = model.predict(future)
    # Plotar a série original
    plt.figure(figsize=(20, 6))
    plt.subplot(2, 1, 1)
    plt.plot(df_pro['ds'], df_pro['y'], label=i, color='blue')
    # Plotar a previsão
    plt.plot(forecast['ds'], forecast['yhat'], label='Forecast', color='red')
    # Adicionar intervalos de incerteza
    plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], color='green', alpha=0.2)
    
    # Configurar o gráfico
    plt.title(var[var['Variable'] == i]['Description'].values[0] )
    plt.xlabel("Sampling")
    plt.ylabel(i)
    plt.grid(True, linestyle='--', color='gray', alpha=0.5)
    plt.legend()

    # Salvar o gráfico da previsão
    plt.savefig(f"../images/{i}_"+"sample.png", dpi=300)

    # Limpar a figura atual para o próximo gráfico
    plt.clf()

    # Segundo gráfico
    plt.figure(figsize=(20, 6))
    plt.plot(df_test['ds'], df_test[i], label=i)
    plt.title(var[var['Variable'] == i]['Description'].values[0] )
    plt.xlabel("Sampling")
    plt.ylabel(i)
    plt.grid(True, linestyle='--', color='gray', alpha=0.5)
    plt.legend()

    # Salvar o gráfico de teste
    plt.savefig(f"../images/{i}_"+"mean.png", dpi=300)

    # Limpar a figura para evitar sobreposição
    plt.clf()


In [ ]:
import pandas as pd
pd.read_csv('../data9724_TAR.csv.gz',compression='gzip')